In [13]:
import pandas as pd
import sys
import glob
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torchvision import models
import torch.nn as nn
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

sys.path.append('finetuning/')
from utils import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
df=pd.DataFrame()
df['image']=glob.glob('./lemon_dataset/*/*.jpg')
df['label']=df.image.str.contains('good')

In [5]:
set_seed(1998,device)

X_train, X_test, y_train, y_test=train_test_split(
    df['image'],df['label'],test_size=0.2,random_state=1998,shuffle=True,
    stratify=df['label'])

X_train, X_val, y_train, y_val=train_test_split(
    X_train,y_train,test_size=0.3,random_state=1998,shuffle=True,
    stratify=y_train)

In [6]:
data_transforms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomResizedCrop(300),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [7]:
train_ds=dataset(X_train,data_transforms)
val_ds=dataset(X_train,data_transforms)
test_ds=dataset(X_test,data_transforms)

train_loader=DataLoader(train_ds,batch_size=8,
shuffle=True,num_workers=0,drop_last=True)

val_loader=DataLoader(val_ds,batch_size=8,
shuffle=True,num_workers=0,drop_last=True)

test_loader=DataLoader(test_ds,batch_size=8,
shuffle=True,num_workers=0,drop_last=True)

In [8]:
pretrained_resnet50 = models.resnet50(pretrained=True).to(device)

for param in pretrained_resnet50.parameters():
    param.requires_grad=False
    
in_features=pretrained_resnet50.fc.in_features
pretrained_resnet50.fc = nn.Linear(in_features, 1).to(device)

c:\Users\icako\anaconda3\envs\Seung\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\icako\anaconda3\envs\Seung\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
#from torchsummary import summary
#summary(pretrained_resnet50, input_size=(3, 300, 300))

In [9]:
from train import *

In [10]:
criterion= nn.BCEWithLogitsLoss()

In [54]:
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
import datetime

time_stamp = str(datetime.datetime.now().timestamp())
log_dir = 'runs/resnet' + time_stamp # runs 경로
#writer = SummaryWriter(log_dir)

#%load_ext tensorboard

# 텐보 로드
#%tensorboard --logdir logs/tensorboard


train(pretrained_resnet50,train_loader,
val_loader,criterion,1,0.00005,device,weight_name='output_trained_once')

100%|██████████| 1/1 [00:21<00:00, 21.58s/it]

Best Trial Renewed

        Train Accuracy is 0.822
        Validation Accuracy is 0.829
        Current best Accuracy is 0.829
        


In [55]:
resnet50 = models.resnet50(pretrained=False).to(device)
resnet50.fc = nn.Linear(resnet50.fc.in_features, 1).to(device)
model=torch.load('./runs/resnet/output_trained_once.pt')
resnet50.fc.weight=nn.Parameter(model['fc.weight']) #Resnet backbone에 출력층 학습된 것만 사용

In [56]:
train(resnet50,train_loader,val_loader,
criterion,50,0.00005,device,weight_name='resnet_fine_tuning')

  2%|▏         | 1/50 [00:35<28:46, 35.24s/it]

Best Trial Renewed

        Train Accuracy is 0.767
        Validation Accuracy is 0.914
        Current best Accuracy is 0.914
        


  4%|▍         | 2/50 [01:06<26:15, 32.83s/it]


        Train Accuracy is 0.783
        Validation Accuracy is 0.903
        Current best Accuracy is 0.914
        


  6%|▌         | 3/50 [01:37<25:12, 32.18s/it]

...


        Train Accuracy is 0.965
        Validation Accuracy is 0.973
        Current best Accuracy is 0.987
        


 98%|█████████▊| 49/50 [25:52<00:30, 30.37s/it]


        Train Accuracy is 0.958
        Validation Accuracy is 0.98
        Current best Accuracy is 0.987
        


100%|██████████| 50/50 [26:21<00:00, 31.62s/it]


        Train Accuracy is 0.978
        Validation Accuracy is 0.981
        Current best Accuracy is 0.987
        


# 출력층을 별도로 학습한 뒤 전체 레이어를 fine-Tuning하는 방안 

In [57]:
resnet50 = models.resnet50(pretrained=False).to(device)
weight=torch.load('./runs/resnet/output_trained_once.pt')

def inference(model,test_loader,weight,device):

    model.fc = nn.Linear(model.fc.in_features, 1).to(device)
    model.load_state_dict(weight)

    test_correct=0
    count=0

    model.eval()
    with torch.no_grad():
        for xx,yy in test_loader:
            xx=xx.to(device)
            yy=yy.to(device)
            output=model(xx)
            predicted=torch.round(torch.sigmoid(output))
            count+=yy.size(0)
            test_correct+=(predicted==yy.unsqueeze(1)).sum().item()

    print(round(test_correct/count,3))

inference(resnet50,test_loader,weight,device)

0.865


# 전체 레이어를 학습하는, resnet의 backbone만 사용하는 fine-Tuning 방안 

In [58]:
weight=torch.load('./runs/resnet/resnet_fine_tuning.pt')
inference(resnet50,test_loader,weight,device)

0.981
